In [130]:
import pandas as pd
import numpy as np
import time
from time import mktime
from datetime import datetime, timedelta
import geopy.distance
import gc
import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

import data_prep_functions
import request_api_nba_stats

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
base_ini = pd.read_csv("../bases_nba_stats/aux_files/base_nba_per_game_16_17.csv")

In [3]:
all_games_16_17 = data_prep_functions.prepara_base(base_ini)

In [4]:
all_games_16_17 = data_prep_functions.cria_features(all_games_16_17)

In [5]:
geral_16_17 = data_prep_functions.gera_last_N_games(all_games_16_17)

In [6]:
data_prep_functions.get_last_games(all_games_16_17, 
                                   geral_16_17[geral_16_17.TEAM_NAME_home == "Pacers"].tail(1).DATE.iloc[0]
                                   , "IND")[["team_home", "team_away", "PTS_home", "PTS_away", 
                                             "FGA_home", "FGA_away", "fl_home_win"]]

,team_home,team_away,PTS_home,PTS_away,FGA_home,FGA_away,fl_home_win
GAME,,,,,,,
IND @ CLE 2017-04-02,CLE,IND,135,130,95,105,1
TOR @ IND 2017-04-04,IND,TOR,108,90,89,76,1
MIL @ IND 2017-04-06,IND,MIL,104,89,80,71,1
IND @ ORL 2017-04-08,ORL,IND,112,127,84,86,0
IND @ PHI 2017-04-10,PHI,IND,111,120,75,85,0


In [7]:
geral_16_17[geral_16_17.TEAM_NAME_home == "Pacers"].tail(1)["FGA_home_L5"]

ATL @ IND 2017-04-12    89.0
Name: FGA_home_L5, dtype: float64

In [8]:
cross_16_17 = data_prep_functions.variaveis_delta(geral_16_17)

In [9]:
cross_16_17.to_csv("../bases_nba_stats/delta_cross_L5_nba_16_17.csv")

In [11]:
cross_16_17.fl_home_win.mean()

0.583739837398374

### BASES PARA PREDICT

In [ ]:
all_games = pd.read_csv("all_games_nba_18_19.csv")

In [137]:
games_ids = request_api_nba_stats.get_list_gameids(datetime.today(), start_at=len(all_games)) 

In [138]:
df_full, df_full_jogo, erros = request_api_nba_stats.get_nba_stats_data(games_ids)

In [139]:
base_nba_2018 = pd.concat(df_full_jogo).reset_index()

In [140]:
new_all_games = data_prep_functions.prepara_base(base_nba_2018)
new_all_games = data_prep_functions.cria_features(new_all_games, all_games)

In [110]:
all_games = pd.concat([all_games, new_all_games])

In [145]:
resp = data_prep_functions.gera_last_N_games(all_games, N=[5])

In [146]:
delta_L5_2018 = data_prep_functions.variaveis_delta(resp)

In [147]:
delta_L5_2018.to_csv("delta_cross_L5_nba_18_19.csv")

In [144]:
all_games.to_csv("all_games_nba_18_19.csv")

### Fluxo Predição no Dia

In [115]:
new_games_ids = request_api_nba_stats.get_list_gameids(max_date = datetime.fromtimestamp(mktime(time.localtime(time.time() + 24*3600)))
                                                   , year = '2018', start_at=len(all_games))

In [132]:
new_games = request_api_nba_stats.get_game_infos_dates(new_games_ids)

In [133]:
new_games = data_prep_functions.cria_features(new_games, all_games)

In [134]:
to_predict = data_prep_functions.gera_last_N_games(new_games, all_games)

In [135]:
to_predict = data_prep_functions.variaveis_delta(to_predict)

In [136]:
to_predict[['D2_PLUS_MINUS_L5', 'D1_WIN_PCT_L5']]

,D2_PLUS_MINUS_L5,D1_WIN_PCT_L5
BOS @ DEN 2018-11-05,9.8,0.0
TOR @ UTA 2018-11-05,-6.2,-0.4
MEM @ GSW 2018-11-05,18.4,0.2
MIN @ LAC 2018-11-05,20.8,0.2
ATL @ CHA 2018-11-06,24.4,0.4
WAS @ DAL 2018-11-06,11.6,0.0
BKN @ PHX 2018-11-06,-1.6,-0.2
MIL @ POR 2018-11-06,4.8,0.0
